In [3]:
import json
import random
from spacy.tokens import DocBin
import spacy

In [4]:
!pip install seqeval 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 619.1 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=1a1ff8b092d93d12c24b19c007aec15ece3448c0f870fd36164e2abdf44b1349
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


# Data Prep

In [6]:
# Function to split data into train and test sets
def split_data(data, train_ratio=0.8):
    random.shuffle(data)
    split_index = int(len(data) * train_ratio)
    return data[:split_index], data[split_index:]

# Load the data from JSON file
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# Function to create DocBin format
def create_docbin(data, output_file):
    doc_bin = DocBin()
    nlp = spacy.blank('id')
    ner = nlp.add_pipe('ner')

    # Add entity labels to the model
    labels = set()
    for annotation in data:
        for entity in annotation['entities']:
            labels.add(entity['label'])
    for label in labels:
        ner.add_label(label)

    # Create DocBin
    for annotation in data:
        doc = nlp.make_doc(annotation['text'])
        ents = []
        for entity in annotation['entities']:
            start, end = entity['start'], entity['end']
            label = entity['label']
            span = doc.char_span(start, end, label=label)
            if span is not None:
                ents.append(span)
        doc.ents = ents
        doc_bin.add(doc)

    doc_bin.to_disk(output_file)

# Load data from JSON file
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# Paths to your files
json_file = '/kaggle/input/tesin-spesi-v2/train_data.json'  # Ganti dengan path file JSON Anda
data = load_json(json_file)

# Split data into training and validation
train_data, valid_data = split_data(data)

# Create DocBin files
train_file = '/kaggle/working/train.spacy'
valid_file = '/kaggle/working/valid.spacy'
create_docbin(train_data, train_file)
create_docbin(valid_data, valid_file)


# Model

In [7]:
!python -m spacy init config config.cfg --lang id --pipeline ner --optimize accuracy

# !python -m spacy init config config.cfg --lang id --pipeline transformer,ner --optimize accuracy --force

# !python -m spacy init config config.cfg --lang id --pipeline ner --optimize accuracy --gpu-id 0

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: id
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [8]:
!python -m spacy train config.cfg --output ./output --paths.train /kaggle/working/train.spacy --paths.dev /kaggle/working/valid.spacy

# !python -m spacy train config.cfg --output ./output --paths.train /kaggle/working/train.spacy --paths.dev /kaggle/working/valid.spacy --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2024-08-21 04:02:13,305] [INFO] Set up nlp object from config
[2024-08-21 04:02:13,357] [INFO] Pipeline: ['tok2vec', 'ner']
[2024-08-21 04:02:13,366] [INFO] Created vocabulary
[2024-08-21 04:02:13,367] [INFO] Finished initializing nlp object
[2024-08-21 04:02:26,397] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     96.00    0.87    0.48    4.13    0.01
  0     200        746.04   3440.03   67.66   75.14   61.54    0.68
  0     400         27.16    858.98   69.42   73.06   66.14    0.69
  0     

# Accuracy

## - Seqeval 

In [9]:
import spacy
from spacy.tokens import DocBin
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2

# Load the trained model once
model_dir = '/kaggle/working/output/model-best'
nlp = spacy.load(model_dir, disable=["tagger", "parser", "attribute_ruler", "lemmatizer"])

# Load validation data
def load_docbin(file_path):
    doc_bin = DocBin().from_disk(file_path)
    return list(doc_bin.get_docs(nlp.vocab))

valid_docs = load_docbin('/kaggle/working/valid.spacy')

# Convert entities to IOB2 format
def convert_to_iob2(docs):
    y_true = []
    y_pred = []

    for doc in docs:
        true_labels = ['O'] * len(doc)
        pred_labels = ['O'] * len(doc)

        # Create a mapping from token index to true labels
        for ent in doc.ents:
            true_labels[ent.start] = 'B-' + ent.label_
            for i in range(ent.start + 1, ent.end):
                true_labels[i] = 'I-' + ent.label_

        # Process the document with the trained model and map predicted labels
        pred_doc = nlp(doc.text)
        for ent in pred_doc.ents:
            pred_labels[ent.start] = 'B-' + ent.label_
            for i in range(ent.start + 1, ent.end):
                pred_labels[i] = 'I-' + ent.label_

        # Append the labels to y_true and y_pred
        y_true.append(true_labels)
        y_pred.append(pred_labels)

    return y_true, y_pred

# Get true and predicted labels
y_true, y_pred = convert_to_iob2(valid_docs)

# Compute classification report
report = classification_report(y_true, y_pred, scheme=IOB2, mode='strict')
print(report)


              precision    recall  f1-score   support

      LOKASI       0.83      0.76      0.79       547
 RUMPUT LAUT       0.89      0.92      0.90       519

   micro avg       0.86      0.84      0.85      1066
   macro avg       0.86      0.84      0.85      1066
weighted avg       0.85      0.84      0.84      1066



## - Nervaluate

In [15]:
!pip install nervaluate

In [62]:
import spacy
from nervaluate import Evaluator
from spacy.training import Example
from spacy.scorer import Scorer

In [67]:
json_file = '/kaggle/input/tesin-spesi-v2/test_data.json'
test_data = load_json(json_file)

# Load the trained model once
model_dir = '/kaggle/working/output/model-best'
nlp = spacy.load(model_dir, disable=["tagger", "parser", "attribute_ruler", "lemmatizer"])

In [68]:
test_data[1]

{'text': 'Sektor pertanian dan perikanan memiliki peranan penting di Indonesia karena kedua sector ini mampu menyediakan lapangan kerja, mampu mendukung sektor industri baik industri hulu maupun industri hilir, mampu menyediakan keragaman menu pangan dan ketersediaan pangan, Salah satu sub system agribisnis adalah produksi sektor ini memegang peranan dalam meningkatkan pemenuhan baik secara kuantitas maupun kualitas produk, rumput laut memiliki potensi ekonomi yang sangat besar dan menjadi salah satu komoditas unggulan di bidang perikanan, dalam pengembangan budidaya rumput laut hal yang lebih penting memperhatikan sisi keberlanjutan dalam kuntinuitas ketersediaan rumput laut. Untuk mendukung hal tersebut diperlukan upaya-upaya dalam pengelolaan budidaya rumput laut salah satunya yaitu pengggunaan kultur jaringan, mengingat sebagian besar petani rumput laut di dalam pelaksanaan budidayanya menggunakan bibit yang masih konvensional dalam artian bahan bibit berasal dari tanaman – tanaman

In [69]:
def extract_entities(data, model):
    entities = []
    for item in data:
        text = item["text"]  # Mengakses 'text' dari dictionary
        doc = model(text)
        entities.append(
            [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
        )
    return entities


# Extract entities from test data
predicted_entities = extract_entities(test_data, nlp)
true_entities = [entity for item in test_data for entity in item['entities']]

In [70]:
true_entities[1]

{'start': 195, 'end': 212, 'label': 'RUMPUT LAUT'}

In [76]:
# Ubah format entitas yang benar (true_entities) ke format yang diharapkan oleh nervaluate
true_entities_formatted = []
for item in test_data:
    true_entities_formatted.append([
        {'start': entity['start'], 'end': entity['end'], 'label': entity['label']}
        for entity in item['entities']
    ])

# Ubah format entitas prediksi ke format yang diharapkan oleh nervaluate
predicted_entities_formatted = []
for doc_entities in predicted_entities:
    predicted_entities_formatted.append([
        {'start': start, 'end': end, 'label': label}
        for start, end, label in doc_entities
    ])

# Inisialisasi Evaluator
evaluator = Evaluator(true_entities_formatted, predicted_entities_formatted, tags=['LOKASI', 'RUMPUT LAUT'])  # Sesuaikan dengan label yang Anda gunakan

# Hitung metrik
results = evaluator.evaluate()  # Perubahan di sini

# Tampilkan hasil
print(results)

({'ent_type': {'correct': 1969, 'incorrect': 13, 'partial': 0, 'missed': 296, 'spurious': 231, 'possible': 2278, 'actual': 2213, 'precision': 0.8897424310890194, 'recall': 0.8643546971027217, 'f1': 0.8768648407926966}, 'partial': {'correct': 1838, 'incorrect': 0, 'partial': 144, 'missed': 296, 'spurious': 231, 'possible': 2278, 'actual': 2213, 'precision': 0.8630817894261184, 'recall': 0.8384547848990342, 'f1': 0.8505900690269427}, 'strict': {'correct': 1827, 'incorrect': 155, 'partial': 0, 'missed': 296, 'spurious': 231, 'possible': 2278, 'actual': 2213, 'precision': 0.8255761409850881, 'recall': 0.8020193151887621, 'f1': 0.8136272545090181}, 'exact': {'correct': 1838, 'incorrect': 144, 'partial': 0, 'missed': 296, 'spurious': 231, 'possible': 2278, 'actual': 2213, 'precision': 0.8305467690917306, 'recall': 0.8068481123792801, 'f1': 0.8185259407704297}}, {'LOKASI': {'ent_type': {'correct': 913, 'incorrect': 2, 'partial': 0, 'missed': 215, 'spurious': 141, 'possible': 1130, 'actual': 1

## Random Test

In [77]:
# Memuat model yang sudah dilatih
model_dir = '/kaggle/working/output/model-best'  # Ganti dengan path model Anda
nlp = spacy.load(model_dir)

# Pilih salah satu data uji
sample_doc = valid_docs[42]  # Anda bisa mengganti indeks sesuai dengan data yang diinginkan

# Teks asli dari data uji
original_text = sample_doc.text

# Buat prediksi menggunakan model
predicted_doc = nlp(original_text)

# Cetak teks asli
print("Original Text:\n", original_text)

# Cetak entitas yang diprediksi oleh model
print("\nPredicted Entities:")
for ent in predicted_doc.ents:
    print(f"Entity: {ent.text}, Label: {ent.label_}, Start: {ent.start_char}, End: {ent.end_char}")


Original Text:
 Paradigrma subsektor perikanan yang selama ini hanya tertumpu pada kegiatan penangkapan hasil-hasil perikanan maka dipandang perlu untuk melakukan suatu kegiatan yang bisa menghasilkan produksi perikanan yaitu dengan cara budidaya. Rumput laut adalah salah satu komoditas perikanan yang sangat potensial untuk dikembangkan dan dapat meningkatkan taraf hidup masyarakat pesisir. Luas lahan di Perairan Kepulauan Banda sangat mendukung pertumbuhan rumput laut, namun lahan ini belum dimanfaatkan secara optimal. Jenis rumput laut yang paling popular dibudidayakan adalah jenis alga merah Kappaphycus alvarezii (Doty) karena memiliki kandungan karagenan yang tinggi. Karagenan digunakan untuk berbagai bidang industry misalnya pengecatan, makanan maupun obatobatan. Pertumbuhan rumput laut terkadang mengalami kendala yaitupertumbuhan yang lambat karena pemilihan metode yang tidak tepat serta diserang penyakit ice-ice. Tujuan dari penelitian ini adalah untuk mengetahui laju pertumbuha

## SKlearn Metrics Accuracy

In [78]:
import json
import spacy
from spacy.training import Example
from sklearn.metrics import classification_report
from collections import defaultdict

In [79]:
# Load the trained model once
model_dir = '/kaggle/working/output/model-best'
nlp = spacy.load(model_dir, disable=["tagger", "parser", "attribute_ruler", "lemmatizer"])

In [81]:
def load_json_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    # Debugging: Periksa format data yang dimuat
    print("Loaded data sample:", data[:1] if data else "No data")
    return data

test_data = load_json_data('/kaggle/input/tesin-spesi-v2/test_data.json')

Loaded data sample: [{'text': 'Indonesia merupakan negara maritime yaitu negara dengan luas perairan yang lebih luas dari pada luas daratan. Potensi perairan yang besar ini telah dimanfaatkan untuk budidaya rumput laut. Jenis Eucheuma Cottonii merupakan penghasil karaginan karena memiliki kadar karaginan yang demikian tinggi, sekitar 62-68% berat keringnya. Karaginan memiliki kegunaan yang sangat banyak baik dalam industri pangan maupun non pangan. Metode yang digunakan dalam pabrik ini adalah Semi-Refined Carrageenan karena proses pembuatannya cepat dan tidak membutuhkan biaya yang banyak. Dilakukan proses yang berurutan yang terbagi menjadi tiga unit proses, yaitu Unit Pre-treatment, Unit Perebusan dalam Alkali, dan Unit Pengolahan Lanjut. Proses pre-treatment adalah pencucian untuk menghilangkan kotoran pada rumput laut dan pemotong untuk memperkecil ukuran rumput laut. Unit Perebusan dalam Alkali bertujuan untuk mendapatkan karaginan yang terkandung pada rumput laut. Selanjutnya ad

In [82]:
print("First item in test data:", test_data[0] if test_data else "No data")

First item in test data: {'text': 'Indonesia merupakan negara maritime yaitu negara dengan luas perairan yang lebih luas dari pada luas daratan. Potensi perairan yang besar ini telah dimanfaatkan untuk budidaya rumput laut. Jenis Eucheuma Cottonii merupakan penghasil karaginan karena memiliki kadar karaginan yang demikian tinggi, sekitar 62-68% berat keringnya. Karaginan memiliki kegunaan yang sangat banyak baik dalam industri pangan maupun non pangan. Metode yang digunakan dalam pabrik ini adalah Semi-Refined Carrageenan karena proses pembuatannya cepat dan tidak membutuhkan biaya yang banyak. Dilakukan proses yang berurutan yang terbagi menjadi tiga unit proses, yaitu Unit Pre-treatment, Unit Perebusan dalam Alkali, dan Unit Pengolahan Lanjut. Proses pre-treatment adalah pencucian untuk menghilangkan kotoran pada rumput laut dan pemotong untuk memperkecil ukuran rumput laut. Unit Perebusan dalam Alkali bertujuan untuk mendapatkan karaginan yang terkandung pada rumput laut. Selanjutny

In [89]:
import json
import spacy
from sklearn.metrics import classification_report
from typing import List, Dict

def load_json_data(file_path: str):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

def extract_entities(doc):
    return [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]

def evaluate_ner_model(nlp, test_data, entity_labels):
    y_true = []
    y_pred = []

    for item in test_data:
        text = item['text']
        true_entities = item.get('entities', [])

        # Create true_labels and pred_labels lists
        true_labels = ['O'] * len(text)
        for entity in true_entities:
            start = entity['start']
            end = entity['end']
            label = entity['label']
            if label in entity_labels:
                for i in range(start, end):
                    true_labels[i] = label

        # Predict using the model
        doc = nlp(text)
        pred_entities = extract_entities(doc)

        pred_labels = ['O'] * len(text)
        for start, end, label in pred_entities:
            if label in entity_labels:
                for i in range(start, end):
                    pred_labels[i] = label

        # Extend the y_true and y_pred lists
        y_true.extend(true_labels)
        y_pred.extend(pred_labels)

    # Generate classification report
    report = classification_report(y_true, y_pred, labels=entity_labels, zero_division=0)
    print(report)

# Example usage
model_dir = '/kaggle/working/output/model-best'
nlp = spacy.load(model_dir, disable=["tagger", "parser", "attribute_ruler", "lemmatizer"])

# Load test data
test_data = load_json_data('/kaggle/input/tesin-spesi-v2/test_data.json')

# Define your entity labels
entity_labels = ['RUMPUT LAUT', 'LOKASI']

# Evaluate the model
evaluate_ner_model(nlp, test_data, entity_labels)


              precision    recall  f1-score   support

 RUMPUT LAUT       0.92      0.90      0.91     16927
      LOKASI       0.86      0.86      0.86     16628

   micro avg       0.89      0.88      0.89     33555
   macro avg       0.89      0.88      0.89     33555
weighted avg       0.89      0.88      0.89     33555



# Save model

In [11]:
# !zip -r output.zip /kaggle/working/output

  adding: kaggle/working/output/ (stored 0%)
  adding: kaggle/working/output/model-last/ (stored 0%)
  adding: kaggle/working/output/model-last/meta.json (deflated 55%)
  adding: kaggle/working/output/model-last/ner/ (stored 0%)
  adding: kaggle/working/output/model-last/ner/moves (deflated 54%)
  adding: kaggle/working/output/model-last/ner/model (deflated 7%)
  adding: kaggle/working/output/model-last/ner/cfg (deflated 33%)
  adding: kaggle/working/output/model-last/tok2vec/ (stored 0%)
  adding: kaggle/working/output/model-last/tok2vec/model (deflated 8%)
  adding: kaggle/working/output/model-last/tok2vec/cfg (stored 0%)
  adding: kaggle/working/output/model-last/tokenizer (deflated 75%)
  adding: kaggle/working/output/model-last/vocab/ (stored 0%)
  adding: kaggle/working/output/model-last/vocab/strings.json (deflated 78%)
  adding: kaggle/working/output/model-last/vocab/vectors (deflated 45%)
  adding: kaggle/working/output/model-last/vocab/lookups.bin (stored 0%)
  adding: kaggle

In [12]:
# from IPython.display import FileLink
# FileLink(r'output.zip')

/kaggle/working/output.zip